# ML Hackathon: HMM and Reinforcement Learning Word Completion

## Problem Statement
This notebook implements:
1. **Part 1**: Hidden Markov Model for letter probability estimation
2. **Part 2**: Reinforcement Learning agent using HMM guidance

### Install Required Packages
Install core dependencies like hmmlearn used for training Hidden Markov Models.

In [ ]:
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 6.5 MB/s eta 0:00:00


---
# Part 1: Hidden Markov Model Implementation

This code defines a Hidden Markov Model (HMM) system that learns letter patterns in words, grouped by their lengths.
It trains a separate HMM for each word length using a text corpus, then predicts the most likely next letter in a masked word by combining HMM emission probabilities with word frequency information.

In [ ]:
import numpy as np, pickle
from hmmlearn.hmm import MultinomialHMM
from collections import defaultdict, Counter

alphabet = "abcdefghijklmnopqrstuvwxyz"
A2I = {c:i for i,c in enumerate(alphabet)}
I2A = {i:c for c,i in A2I.items()}
N = len(alphabet)

# Convert word → sequence of one-hot vectors (expected by new hmmlearn)
def encode_word(word):
    seq = np.zeros((len(word), N), dtype=int)
    for i, c in enumerate(word):
        seq[i, A2I[c]] = 1
    return seq


class PerLengthHMMs:
    def __init__(self):
        self.hmm_by_length = {}
        self.words_by_length = {}

    def save(self, path):
        with open(path, "wb") as f:
            pickle.dump(self, f)
        print(f"✅ Saved model to {path}")

    @staticmethod
    def load(path):
        with open(path, "rb") as f:
            print(f"✅ Loaded model from {path}")
            return pickle.load(f)

    def train(self, corpus_path, states_per_pos=3, max_len=25):
        print("📌 Loading corpus...")
        with open(corpus_path, 'r') as f:
            words = [w.strip().lower() for w in f if w.strip().isalpha()]

        print(f"📌 {len(words)} words loaded.")

        # group by length
        for w in words:
            if len(w) <= max_len:
                self.words_by_length.setdefault(len(w), []).append(w)

        for L, word_list in sorted(self.words_by_length.items()):
            print(f"\n🎯 Training HMM for length {L} ({len(word_list)} words)...")

            n_states = max(2, states_per_pos * L)

            model = MultinomialHMM(
                n_components=n_states,
                n_iter=15,
                tol=1e-4,
                init_params="",
                params="ste"
            )

            model.n_features = N

            # Initialize parameters
            model.startprob_ = np.full(n_states, 1/n_states)
            model.transmat_  = np.full((n_states, n_states), 1/n_states)
            model.emissionprob_ = np.full((n_states, N), 1/N)

            # Build training matrix
            X = np.vstack([encode_word(w) for w in word_list])
            lengths = [len(w) for w in word_list]

            model.fit(X, lengths)
            self.hmm_by_length[L] = model

        print("\n✅ Training complete!")

    def _candidate_words(self, masked, guessed):
        L = len(masked)
        if L not in self.words_by_length:
            return []
        return [
            w for w in self.words_by_length[L]
            if all((m == "_" or w[i] == m) and (w[i] not in guessed or m == w[i])
                   for i, m in enumerate(masked))
        ]

    def predict_letter_distribution(self, masked, guessed):
        L = len(masked)
        if L not in self.hmm_by_length:
            return None

        hmm = self.hmm_by_length[L]

        # HMM-based letter weights (average emission)
        p_hmm = hmm.emissionprob_.mean(axis=0)

        for g in guessed:
            p_hmm[A2I[g]] = 0

        # Normalize
        s = p_hmm.sum()
        if s > 0:
            p_hmm = p_hmm / s
        else:
            p_hmm = np.ones(N) / N

        p_hmm = {I2A[i]: p_hmm[i] for i in range(N)}

        # Word frequency weighting
        candidates = self._candidate_words(masked, guessed)
        freq = Counter()
        for w in candidates:
            for i, c in enumerate(masked):
                if c == "_":
                    freq[w[i]] += 1
        total = sum(freq.values()) + 1e-9
        p_freq = {c: freq[c]/total for c in alphabet}

        # Blend both
        ALPHA = 0.6
        final = {c: ALPHA*p_hmm.get(c,0) + (1-ALPHA)*p_freq.get(c,0) for c in alphabet}

        # Normalize final distribution
        S = sum(final.values()) + 1e-9
        final = {c: final[c]/S for c in final}

        return final

    def predict_letter(self, masked, guessed):
        dist = self.predict_letter_distribution(masked, guessed)
        return max(dist.items(), key=lambda x: x[1])[0]


This code creates an instance of the PerLengthHMMs class, trains it on the given word corpus to learn HMMs for different word lengths, and then saves the trained model to a file for later use.

In [ ]:
model = PerLengthHMMs()
model.train("/content/corpus.txt")
model.save("/content/per_length_hmms.pkl")


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


📌 Loading corpus...
📌 45701 words loaded.

🎯 Training HMM for length 1 (23 words)...

🎯 Training HMM for length 2 (33 words)...

🎯 Training HMM for length 3 (246 words)...


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340



🎯 Training HMM for length 4 (937 words)...


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


...


🎯 Training HMM for length 21 (17 words)...


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340



🎯 Training HMM for length 22 (8 words)...


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340



🎯 Training HMM for length 23 (3 words)...

🎯 Training HMM for length 24 (1 words)...

✅ Training complete!
✅ Saved model to /content/new_per_length_hmms.pkl


---
# Part 2: Reinforcement Learning Implementation

This code builds and trains a Deep Q-Network (DQN) reinforcement learning agent to play Hangman using the patterns learned by the HMM.
It loads the pre-trained HMM to guide letter prediction, defines a game environment and reward system, and then trains the DQN over thousands of episodes to maximize correct guesses and minimize wrong ones.

In [ ]:
# As the HMM has been trained now we are focusing on building the RL using DQN.
!pip install torch torchvision

import random
from collections import deque
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from tqdm import tqdm

# =====================
# CONFIG
# =====================
MODEL_PATH = "/content/per_length_hmms.pkl"
MAX_LEN = 25
MAX_LIVES = 6
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

NUM_EPISODES = 10000
BATCH_SIZE = 128
GAMMA = 0.99
LR = 1e-4
BUFFER_CAPACITY = 150000
TARGET_UPDATE_FREQ = 600
EPS_START = 1.0
EPS_END = 0.05
EPS_DECAY = 0.9993
HIDDEN = 256

# =====================
# LOAD HMM
# =====================
with open(MODEL_PATH, "rb") as f:
    trainer = pickle.load(f)

print("✅ Loaded HMM trainer")

alphabet = "abcdefghijklmnopqrstuvwxyz"
V = len(alphabet)
A2I = {c:i for i,c in enumerate(alphabet)}
I2A = {i:c for c,i in A2I.items()}

def get_hmm_letter_agg(masked, guessed, trainer):
    dist = trainer.predict_letter_distribution(masked, guessed)
    if dist is None:
        vec = np.ones(V)/V
    else:
        vec = np.array([dist.get(c,0) for c in alphabet], dtype=np.float32)
    for g in guessed:
        if g in A2I:
            vec[A2I[g]] = 0
    s = vec.sum()
    return (vec/s if s>0 else np.ones(V)/V).astype(np.float32)

# =====================
# ENVIRONMENT
# =====================
class HangmanEnv:
    def __init__(self, trainer, max_len=MAX_LEN, max_lives=MAX_LIVES):
        self.trainer = trainer
        self.max_len = max_len
        self.max_lives = max_lives

    def reset(self, word=None):
        if word:
            self.word = word
        else:
            L = random.choice(list(self.trainer.words_by_length.keys()))
            self.word = random.choice(self.trainer.words_by_length[L])
        self.masked = "_" * len(self.word)
        self.guessed = set()
        self.wrong = 0
        return self._get_state()

    def _get_state(self):
        pos = np.zeros(self.max_len * V, np.float32)
        for i, ch in enumerate(self.masked):
            if ch != "_":
                pos[i*V + A2I[ch]] = 1
        guessed_vec = np.zeros(V, np.float32)
        for g in self.guessed:
            guessed_vec[A2I[g]] = 1
        hmm_vec = get_hmm_letter_agg(self.masked, self.guessed, self.trainer)
        life = np.array([(self.max_lives - self.wrong) / self.max_lives], np.float32)
        return np.concatenate([pos, guessed_vec, hmm_vec, life])

    def valid_actions_mask(self):
        m = np.ones(V, bool)
        for g in self.guessed:
            m[A2I[g]] = False
        return m

    def step(self, action_idx):
        letter = I2A[action_idx]
        reward = 0
        done = False

        # Reward policy which has been implemented (Reviewer keep this in mind)
        if letter in self.guessed:
            reward -= 3
        else:
            self.guessed.add(letter)
            if letter in self.word:
                self.masked = "".join(
                    letter if self.word[i] == letter else self.masked[i]
                    for i in range(len(self.word)))
                reward += 2
                if "_" not in self.masked:
                    reward += 10
                    done = True
            else:
                self.wrong += 1
                reward -= 1
                if self.wrong >= self.max_lives:
                    reward -= 10
                    done = True

        return self._get_state(), reward, done, {}

# =====================
# DQN
# =====================
state_dim = MAX_LEN*V + V + V + 1
action_dim = V

class DQN(nn.Module):
    def __init__(self, s, a):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(s, HIDDEN), nn.ReLU(),
            nn.Linear(HIDDEN, HIDDEN), nn.ReLU(),
            nn.Linear(HIDDEN, a))
    def forward(self, x): return self.net(x)

class ReplayBuffer:
    def __init__(self, cap): self.buf = deque(maxlen=cap)
    def push(self, *ex): self.buf.append(ex)
    def sample(self, n):
        batch = random.sample(self.buf, n)
        return map(np.stack, zip(*batch))
    def __len__(self): return len(self.buf)

env = HangmanEnv(trainer)
policy = DQN(state_dim, action_dim).to(DEVICE)
target = DQN(state_dim, action_dim).to(DEVICE)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=LR)
buffer = ReplayBuffer(BUFFER_CAPACITY)

eps = EPS_START
steps = 0

print("🚀 Training DQN...")

for ep in range(NUM_EPISODES):
    state = env.reset()
    done = False
    while not done:
        mask = env.valid_actions_mask()

        if random.random() < eps:
            action = random.choice(np.where(mask)[0])
        else:
            s = torch.tensor(state, dtype=torch.float32, device=DEVICE).unsqueeze(0)
            q = policy(s).detach().cpu().numpy().squeeze()
            q[~mask] = -1e9
            action = int(np.argmax(q))

        next_state, reward, done, _ = env.step(action)
        buffer.push(state, action, reward, next_state, done)
        state = next_state

        if len(buffer) >= BATCH_SIZE:
            s,a,r,s2,d = buffer.sample(BATCH_SIZE)
            s = torch.tensor(s, dtype=torch.float32, device=DEVICE)
            a = torch.tensor(a, dtype=torch.long, device=DEVICE).unsqueeze(1)
            r = torch.tensor(r, dtype=torch.float32, device=DEVICE).unsqueeze(1)
            s2 = torch.tensor(s2, dtype=torch.float32, device=DEVICE)
            d = torch.tensor(d, dtype=torch.float32, device=DEVICE).unsqueeze(1)

            q_vals = policy(s).gather(1, a)
            with torch.no_grad():
                q_next = target(s2)
                guessed = s2[:, MAX_LEN*V:MAX_LEN*V+V]
                mask2 = (guessed < 0.5)
                q_next[~mask2] = -1e9
                max_q = q_next.max(dim=1, keepdim=True)[0]
                target_q = r + (1 - d) * GAMMA * max_q

            loss = nn.functional.mse_loss(q_vals, target_q)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        steps += 1
        eps = max(EPS_END, eps * EPS_DECAY)

        if steps % TARGET_UPDATE_FREQ == 0:
            target.load_state_dict(policy.state_dict())

    if ep % 200 == 0:
        print(f"Ep {ep}/{NUM_EPISODES} | eps={eps:.3f}")

torch.save(policy.state_dict(), "dqn_hangman_policy.pth")
print("✅ Training Complete — Saved: dqn_hangman_policy.pth")
# As mentioned the same policy will be used later to get the total marks for the test set

Using device: cuda
✅ Loaded HMM trainer
🚀 Training DQN...
Ep 0/10000 | eps=0.994
Ep 200/10000 | eps=0.216
Ep 400/10000 | eps=0.050
Ep 600/10000 | eps=0.050
Ep 800/10000 | eps=0.050
Ep 1000/10000 | eps=0.050
Ep 1200/10000 | eps=0.050
Ep 1400/10000 | eps=0.050
Ep 1600/10000 | eps=0.050
Ep 1800/10000 | eps=0.050
Ep 2000/10000 | eps=0.050
Ep 2200/10000 | eps=0.050
Ep 2400/10000 | eps=0.050
Ep 2600/10000 | eps=0.050
Ep 2800/10000 | eps=0.050
Ep 3000/10000 | eps=0.050
Ep 3200/10000 | eps=0.050
Ep 3400/10000 | eps=0.050
Ep 3600/10000 | eps=0.050
Ep 3800/10000 | eps=0.050
Ep 4000/10000 | eps=0.050
Ep 4200/10000 | eps=0.050
Ep 4400/10000 | eps=0.050
Ep 4600/10000 | eps=0.050
Ep 4800/10000 | eps=0.050
Ep 5000/10000 | eps=0.050
Ep 5200/10000 | eps=0.050
Ep 5400/10000 | eps=0.050
Ep 5600/10000 | eps=0.050
Ep 5800/10000 | eps=0.050
Ep 6000/10000 | eps=0.050
Ep 6200/10000 | eps=0.050
Ep 6400/10000 | eps=0.050
Ep 6600/10000 | eps=0.050
Ep 6800/10000 | eps=0.050
Ep 7000/10000 | eps=0.050
Ep 7200/10000

This function evaluates the trained DQN Hangman agent on a set of test words to measure its performance.
It simulates games for each word, logs the guesses and results for a few examples, and finally reports metrics like win rate, average wrong guesses, and total reward.

In [ ]:
from tqdm import tqdm
import numpy as np
import torch

def evaluate_and_show(policy, trainer, test_path, max_show=15):
    env = HangmanEnv(trainer)
    policy.eval()

    with open(test_path) as f:
        test_words = [w.strip() for w in f if len(w.strip()) > 0]

    wins = 0
    total_wrong = 0
    total_repeats = 0
    total_reward = 0

    print(f"\n🔍 Evaluating on {len(test_words)} words...")
    print("Showing first", max_show, "words:\n")

    for idx, word in enumerate(test_words):
        state = env.reset(word=word)
        done = False
        episode_reward = 0

        if idx < max_show:
            print("----------------------------------------------------")
            print(f"🔤 Target Word: {word}")
            print(f"Start: {env.masked}")

        while not done:
            mask = env.valid_actions_mask()

            s = torch.tensor(state, dtype=torch.float32, device=DEVICE).unsqueeze(0)
            q = policy(s).cpu().detach().numpy().squeeze()
            q[~mask] = -1e9
            action = int(np.argmax(q))

            next_state, reward, done, info = env.step(action)
            state = next_state
            episode_reward += reward

            if idx < max_show:
                print(f"Guess: {I2A[action]} | Masked: {env.masked} | Wrong: {env.wrong}", end="")
                if info.get("repeated", False):
                    print("  (repeated)", end="")
                print()

            if info.get("repeated", False):
                total_repeats += 1

        if env.masked == env.word:
            wins += 1
        total_wrong += env.wrong
        total_reward += episode_reward

    # Final Stats
    print("\n==============================")
    print(f"Test Words: {len(test_words)}")
    print(f"Win Rate: {wins/len(test_words)*100:.2f}%")
    print(f"Avg Wrong Guesses: {total_wrong/len(test_words):.2f}")
    print(f"Final Score (sum of rewards): {total_reward:.2f}") # from the previous policy
    print("==============================")


This code loads the previously trained DQN model and runs it on a test dataset to evaluate its performance..

In [ ]:
policy = DQN(state_dim, action_dim).to(DEVICE)
policy.load_state_dict(torch.load("/content/dqn_hangman_policy.pth"))

evaluate_and_show(policy, trainer, "/content/test.txt", max_show=15)


🔍 Evaluating on 2000 words...
Showing first 15 words:

----------------------------------------------------
🔤 Target Word: marmar
Start: ______
Guess: e | Masked: ______ | Wrong: 1
Guess: s | Masked: ______ | Wrong: 2
Guess: a | Masked: _a__a_ | Wrong: 2
Guess: r | Masked: _ar_ar | Wrong: 2
Guess: m | Masked: marmar | Wrong: 2
----------------------------------------------------
🔤 Target Word: janet
Start: _____
Guess: e | Masked: ___e_ | Wrong: 0
Guess: r | Masked: ___e_ | Wrong: 1
Guess: s | Masked: ___e_ | Wrong: 2
Guess: a | Masked: _a_e_ | Wrong: 2
Guess: t | Masked: _a_et | Wrong: 2
Guess: c | Masked: _a_et | Wrong: 3
Guess: w | Masked: _a_et | Wrong: 4
Guess: n | Masked: _anet | Wrong: 4
Guess: k | Masked: _anet | Wrong: 5
Guess: z | Masked: _anet | Wrong: 6
----------------------------------------------------
🔤 Target Word: dentistical
Start: ___________
Guess: t | Masked: ___t__t____ | Wrong: 0
Guess: e | Masked: _e_t__t____ | Wrong: 0
Guess: s | Masked: _e_t_st____ | Wrong: 